In [1]:
#setup based on this: https://t-redactyl.io/blog/2020/08/reading-s3-data-into-a-spark-dataframe-using-sagemaker.html
import boto3
import json 
import time
import pandas as pd
from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *
import matplotlib.pyplot as plt
import sagemaker_pyspark
import botocore.session

## Set Spark Session Configuration

In [2]:
session = botocore.session.get_session()
credentials = session.get_credentials()

In [3]:
client = boto3.client('secretsmanager')
response = client.get_secret_value(
    SecretId='sapient-s3-access'
)
response = json.loads(response['SecretString'])
access_key = response["aws_access_key_id"]
secret_key = response["aws_secret_access_key"]

In [4]:
conf = (SparkConf()
        .set("spark.driver.extraClassPath", ":".join(sagemaker_pyspark.classpath_jars()))
       )

In [5]:
# https://spark.apache.org/docs/latest/configuration.html#memory-management
spark = (
    SparkSession
    .builder
    .config(conf=conf) \
    .config('fs.s3a.access.key', access_key)
    .config('fs.s3a.secret.key', secret_key)
    .config('spark.network.timeout', 300)
    .config('spark.local.dir', '/home/ec2-user/SageMaker/tmp')
    .config("spark.executor.memory", "400g")
    .config("spark.driver.memory", "80g")
    .config("spark.memory.offHeap.enabled", "true")
    .config("spark.memory.offHeap.size","50g")
    .appName("sapient")
    .getOrCreate()
)
spark.sparkContext.setLogLevel("ERROR")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/06 06:31:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/03/06 06:31:40 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


## Functions to Load and Read Data

In [6]:
# read from raw bucket + write to refined bucket + aggregate final to the trusted bucket
s3_url_raw = "s3a://sapient-bucket-raw/"
s3_url_refined = "s3a://sapient-bucket-refined/"
s3_url_trusted = "s3a://sapient-bucket-trusted/"
bro_cols_conn = ['ts', 'uid', 'id.orig_h', 'id.orig_p', 'id.resp_', 'id.resp_p', 'proto', 'service', 'duration', 'orig_bytes', 'resp_bytes', 'conn_state', 
                 'local_orig', 'local_resp', 'missed_bytes', 'history', 'orig_pkts', 'orig_ip_bytes', 'resp_pkts', 'resp_ip_bytes', 'tunnel_parents']
bro_cols_rep = ['ts', 'level', 'message', 'location']

In [10]:
def refined_to_trusted(type='ecar-bro', env='prod', size='small'):
    """
    type: ecar, ecar-bro, bro
    """
    if size == 'small':
        # 100 million
        read_lim = 10000000
    elif size == 'medium':
        # 500 million
        read_lim = 100000000
    elif size == 'large':
        # 1 billion
        read_lim = 1000000000  
    min_cols = ['id','timestamp','objectID','actorID','object','action','hostname', 'user_name']
    if type == 'labels':
        s3_read_loc = f"{s3_url_refined}/{env}/{type}"
        s3_write_loc = f"{s3_url_trusted}/{env}/{type}"
        df = spark.read.parquet(s3_read_loc).cache()
    else:
        s3_read_loc = f"{s3_url_refined}/{env}/{type}/"
        s3_write_loc = f"{s3_url_trusted}/{env}/{type}/{size}"
        start_time = time.time()
        df = spark.read.parquet(s3_read_loc)\
                    .select(*min_cols) \
                    .filter((dayofmonth(col('timestamp')) == 23)) \
                    .cache()
        df = df.limit(read_lim)
    print(time.strftime('%l:%M%p %Z on %b %d, %Y') + " --- read and cache time: %s seconds ---" % (time.time() - start_time))
    start_time = time.time()
    df.write.option("maxRecordsPerFile", 300000).mode("overwrite").parquet(s3_write_loc)
    print(time.strftime('%l:%M%p %Z on %b %d, %Y') + " --- write time: %s seconds ---" % (time.time() - start_time))

In [ ]:
refined_to_trusted('ecar', env=env, size='small')